In [1]:
%reset -f

In [2]:
import ee
ee.Authenticate()


In [ ]:

var prism_ic = ee.ImageCollection('OREGONSTATE/PRISM/AN81m');
var first_im = prism_ic.first().select('ppt');
var scale = first_im.projection().nominalScale().getInfo();

var cover_ic = ee.ImageCollection('projects/rap-data-365417/assets/vegetation-cover-v3');
var prod_ic = ee.ImageCollection('projects/rap-data-365417/assets/npp-partitioned-v3');

var years_list= ee.List([1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005]);
var label_order = ee.List(['RAP_afg', 'RAP_pfg', 'RAP_bgr', 'RAP_ltr', 'RAP_shr', 'RAP_tre', 'RAP_afgNPP', 'RAP_pfgNPP', 'RAP_shrNPP', 'RAP_treNPP']);
var area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', false);

var im = cover_ic.first();
var proj = im.projection().getInfo();

var transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
];

var transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
];

var proj = im.projection();

//RAP STUFF

function clip_fnA(yrobj){
  var year = ee.Number(yrobj);
  var start = ee.Date.fromYMD(year, 1, 1);
  var end = ee.Date.fromYMD(year.add(1), 1, 1);
  var yr_im = cover_ic.filterDate(start, end).select(['AFG', 'PFG', 'BGR', 'LTR', 'SHR', 'TRE']).first();
  var im = yr_im.setDefaultProjection('EPSG:4326', transform);
  var im = im.reproject({crs:proj.crs(), crsTransform:transform_new});
  var clip_im = ee.Image(im).clip(area_shp);
  return clip_im;
}

var c_ic = ee.ImageCollection(years_list.map(clip_fnA));

function clip_fnB(yrobj){
  var year = ee.Number(yrobj);
  var start = ee.Date.fromYMD(year, 1, 1);
  var end = ee.Date.fromYMD(year.add(1), 1, 1);
  var yr_im = prod_ic.filterDate(start, end).select(['afgNPP', 'pfgNPP', 'shrNPP', 'treNPP']).first();
  var im = yr_im.setDefaultProjection('EPSG:4326', transform);
  var im = im.reproject({crs:proj.crs(), crsTransform:transform_new});
  var clip_im = ee.Image(im).clip(area_shp);
  return clip_im;
}

var p_ic = ee.ImageCollection(years_list.map(clip_fnB));



var cavg_im = c_ic.reduce(ee.Reducer.mean());
var afg_im = cavg_im.select('AFG_mean');
var pfg_im = cavg_im.select('PFG_mean');
var bgr_im = cavg_im.select('BGR_mean');
var ltr_im = cavg_im.select('LTR_mean');
var shr_im = cavg_im.select('SHR_mean');
var tre_im = cavg_im.select('TRE_mean');

var pavg_im = p_ic.reduce(ee.Reducer.mean());
var afgnpp_im = pavg_im.select('afgNPP_mean');
var pfgnpp_im = pavg_im.select('pfgNPP_mean');
var shrnpp_im = pavg_im.select('shrNPP_mean');
var trenpp_im = pavg_im.select('treNPP_mean');


var map_ic = ee.ImageCollection([afg_im, pfg_im, bgr_im, ltr_im, shr_im, tre_im, afgnpp_im, pfgnpp_im, shrnpp_im, trenpp_im]);
var map_ic_list = map_ic.toList(map_ic.size());

for (var i = 0; i < map_ic.size().getInfo(); i++){

  var eimage = ee.Image(map_ic_list.get(i));
  var label = ee.String(label_order.get(i)).getInfo();

  Export.image.toDrive({
    image:eimage,
    description:label,
    folder:'GEE_Downloads',
    region:area_shp,
    scale:scale,
    crs:'EPSG:4326',
    maxPixels:1e13
  });
}


